In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import TFBertTokenizer,BertConfig,BertTokenizer
from transformers import AdamW,TFBertForSequenceClassification,get_linear_schedule_with_warmup

In [ ]:
from tqdm import tqdm, trange

In [ ]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import requests

file_url = "https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-2nd-Edition/main/Chapter03/in_domain_train.tsv"
file_name = "in_domain_train.tsv"

response = requests.get(file_url)
with open(file_name, "wb") as file:
    file.write(response.content)

print("File downloaded successfully.")

File downloaded successfully.


In [ ]:
import requests

file_url = "https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-2nd-Edition/main/Chapter03/out_of_domain_dev.tsv"
file_name = "out_of_domain_dev.tsv"

response = requests.get(file_url)
with open(file_name, "wb") as file:
    file.write(response.content)

print("File downloaded successfully.")

File downloaded successfully.


In [ ]:
import pandas as pd

df= pd.read_csv("in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.shape

(8551, 4)

In [ ]:
df.sample(10)

,sentence_source,label,label_notes,sentence
6342,c_13,1,NaN,john believes himself to be the best at baseba...
4803,ks08,1,NaN,lee asked me how fond of chocolates the monkey...
5030,ks08,0,*,kim is eager to recommend .
7138,sks13,0,*,her will put a picture of bill on your desk be...
492,bc01,1,NaN,john forgot .
6811,m_02,1,NaN,jane and elizabeth were both at home .
519,bc01,1,NaN,the box contains the ball .
5015,ks08,1,NaN,this doll is hard to see .
6624,m_02,1,NaN,there are fewer trucks on the motorway this wi...
4662,ks08,1,NaN,the ball was kicked by him .


## Creating sentences, label lists, and adding BERT tokens

In [ ]:
#@ Creating sentence, label lists and adding Bert tokens
sentences=df.sentence.values

# Adding CLS and SEP tokens at the beginning and end of each sentence for BERT

# sentences=["[CLS] "+sentence+" [SEP]" for sentence in sentences]
labels=df.label.values

In [ ]:
labels[:3]

array([1, 1, 1])

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(df['sentence'], df['label'], test_size=0.2, random_state=42)

# Verify the shapes of the train and test sets
print("Train sentences shape:", train_sentences.shape)
print("Test sentences shape:", test_sentences.shape)
print("Train labels shape:", train_labels.shape)
print("Test labels shape:", test_labels.shape)


Train sentences shape: (6840,)
Test sentences shape: (1711,)
Train labels shape: (6840,)
Test labels shape: (1711,)


## Activating BERT tokenizer

In [ ]:
!pip install tensorflow_text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Processing the data


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification,AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preprocessing the dataset

In [ ]:
train_sentences=tokenizer(list(train_sentences),padding='max_length',max_length=128,truncation=True,return_tensors='tf')
test_sentences=tokenizer(list(test_sentences),padding='max_length',max_length=128,truncation=True,return_tensors='tf')


## Preprocessing dataset

In [ ]:
BATCH_SIZE=32
train_data = tf.data.Dataset.from_tensor_slices((dict(train_sentences), train_labels)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((dict(test_sentences), test_labels)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
from sklearn.utils import validation
history = model.fit(train_data,
                    epochs=5,
                    batch_size=BATCH_SIZE,
                    validation_data=test_data
                    )

Epoch 1/5
214/214 [==============================] - 242s 885ms/step - loss: 0.6155 - accuracy: 0.7015 - val_loss: 0.5974 - val_accuracy: 0.7154
Epoch 2/5
 90/214 [===========>..................] - ETA: 1:38 - loss: 0.6135 - accuracy: 0.7031

In [ ]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

history = model.fit(train_dataset,
                    epochs=5,
                    validation_data=test_dataset)


Epoch 1/5


ValueError: ignored